In [32]:
from CoolProp.CoolProp import PropsSI

import numpy as np
from scipy.integrate import solve_ivp
from scipy.misc import derivative
from matplotlib import pyplot as plt

#### application constants
FFMPEG_PATH = r'C:\CLI-Applications\ffmpeg-build\bin\ffmpeg.exe'
MPL_STYLESHEET = r'C:\Users\lnick\Documents\Personal\Programming\Python\Resources\proplot_style.mplstyle'

#### simulation constants

# chemistry
A = 3.274e7         # Arrhenius constant in rate equation               [mol^-1 dm^3 s^-1]
E_A = 5.0e4         # activation energy for reaction in rate equation   [J]
DH = -2.632e4       # enthalpy change for reaction                      [J mol^-1]
R = 8.314           # gas constant                                      [J mol^-1 K^-1]
K = lambda T: A * np.exp(-E_A / (R * T))

# solvent
RHO = lambda T: 1e-3 * PropsSI('D', 'T|liquid', T, 'P', 1e5, 'Water')   # solvent density
D_RHO_DT = lambda T: derivative(RHO, T, 1e-3)                           # [kg dm^-3]
C_P = lambda T: PropsSI('C', 'T|liquid', T, 'P', 1e5, 'Water')          # solvent SHC
D_CP_DT = lambda T: derivative(C_P, T, 1e-3)                            # [J kg^-1 K^-1]

# flow
V = 120             # volume of vessel, fixed                           [dm^3 = litres]
Q = 20              # total flow rate                                   [dm^3 s^-1]
UA = 880 * 3.95     # steam heater overall HTC * surface area           [W K^-1]
T_S = 373           # steam heater temperature                          [K]

# manipulated vars
Q_A = 10            # inflow rate of A                                  [dm^3 s^-1]
Q_B = Q - Q_A       # inflow rate of B                                  [dm^3 s^-1]
C_A1 = 1.5          # concentration of A at inflow                      [mol dm^-3]
C_B1 = 1.8          # concentration of B at inflow                      [mol dm^-3]
T_A1 = 300          # temperature of inflow of A                        [K]
T_B1 = 320          # temperature of inflow of A                        [K]

# stylesheet
plt.style.use(MPL_STYLESHEET)

In [54]:
def ode_system(t: float, y: np.ndarray,
        const_rho: bool = False, const_shc: bool = True, const_rate: bool = False) -> np.ndarray:

    c_A2, c_B2, c_C2, T_2 = y
    k = 0.05584 if const_rate else K(T_2)        # rate constant            [mol^-1 dm^3 s^-1]
    rho = 1.000 if const_rho else RHO(T_2)       # density                  [kg dm^-3]
    c_p = 4200 if const_shc else C_P(T_2)        # isobaric SHC             [J kg^-1 K^-1]
    dcp_dt = 0 if const_shc else D_CP_DT(T_2)    # temperature derivative   [J kg^-1 K^-2]
    drho_dt = 0 if const_rho else D_RHO_DT(T_2)  # temperature derivative   [kg dm^-3 K^-1]
    energy_balance_numer = rho * c_p * (Q_A * T_A1 + Q_B * T_B1 - Q * T_2) \
        - k * V * DH * c_A2 * c_B2 + UA * (T_S - T_2)                     # [J s^-1]
    energy_balance_denom = rho * V * (c_p + T_2 * dcp_dt) \
        + c_p * V * drho_dt                                               # [J K^-1]
    
    return np.array([
        C_A1 * Q_A / V - c_A2 * Q / V - k * c_A2 * c_B2,
        C_B1 * Q_B / V - c_B2 * Q / V - k * c_A2 * c_B2,
        -c_C2 * Q / V + k * c_A2 * c_B2,
        energy_balance_numer / energy_balance_denom
    ])


def solve_sim(init_vals: tuple = [0, 0, 0, 300], t_end: int = 60, plot: bool = False,
        assumptions=(True, True, True)):
    
    sol = solve_ivp(ode_system, (0, t_end), init_vals, args=assumptions)

    if plot:
        plt.title(r'Reaction: $ A + B \rightarrow C $')
        fig, ax = plt.subplots()
        #ax.plot(sol.t, sol.y[0], label=r'$C_{A2}$')
        #ax.plot(sol.t, sol.y[1], label=r'$C_{B2}$')
        ax.plot(sol.t, sol.y[2], label=r'$[C]$')
        ax.set_xlabel('time / s')
        ax.set_ylabel(r'concentration / $ mol \ dm^{-3} $')
        plt.legend(loc='upper left')
        ax2 = ax.twinx()
        ax2.plot(sol.t, sol.y[3], label=r'$T_{2}$', color='r')
        ax2.set_ylabel(r'temperature / K')
        plt.legend(loc='lower right')
        plt.show()

    return (sol.y[2][-1], sol.y[3][-1])

In [53]:
import itertools


for combo in itertools.product((True, False), repeat=3):
    c, T = solve_sim(plot=False, assumptions=combo)
    print(c, T, combo)

0.1504864102589652 313.4082093380277 (True, True, True)
0.27923782157129745 314.1833375381252 (True, True, False)
0.15048640988549902 313.42428847704787 (True, False, True)
0.2794131810584978 314.20403044349644 (True, False, False)
0.15048641026757828 313.43424262378755 (False, True, True)
0.2795295560677017 314.2182960322611 (False, True, False)
0.15048640989212006 313.45045790164494 (False, False, True)
0.27970667323124687 314.23919448621723 (False, False, False)
